In [9]:
import pandas as pd
from pandas import json_normalize
import json

Extração de JSON tem 3 níveis de complexidade:

Nível 1 – JSON simples (um único produto)

Nível 2 – JSON aninhado (produto com histórico de preços em lista)

Nível 3 – JSON mais complexo (pedido com múltiplos produtos e preços promocionais)

In [7]:
# Nível 1

data = [{
  "id": 101,
  "produto": "Notebook",
  "preco": 3500,
  "quantidade": 2
    },
    {
  "id": 102,
  "produto": "TV",
  "preco": 1000,
  "quantidade": 20
    }
]

df = pd.DataFrame(data)
df.head()


,id,produto,preco,quantidade
0,101,Notebook,3500,2
1,102,TV,1000,20


In [ ]:
# Nível 2 = um produto, com uma lista de preços → 1 normalização basta.

data = {
  "produto": {
    "id": 202,
    "nome": "Smartphone",
    "precos": [
      {"data": "2025-01-01", "valor": 2000},
      {"data": "2025-02-01", "valor": 2100}
    ]
  }
}

df = json_normalize(
    data,
    # caminho dentro do JSON até a lista que você quer transformar em linhas da tabela.
    record_path=['produto', 'precos'],
    # outras informações do nível superior que você também quer carregar junto.
    meta=[['produto', 'id'], ['produto', 'nome']]
)

df.head()

,data,valor,produto.id,produto.nome
0,2025-01-01,2000,202,Smartphone
1,2025-02-01,2100,202,Smartphone


In [ ]:
# Nível 3 = um pedido, com vários itens, e cada item tem sua própria lista de preços → precisa de 2 normalizações (ou passos extras).

data = {
    "pedido": {
        "id": 303,
        "cliente": "Carlos",
        "itens": [
            {"produto": "Notebook", "quantidade": 1, "precos": [3500, 3300]},
            {"produto": "Mouse", "quantidade": 2, "precos": [80, 70]}
        ]
    }
}

# 1ª normalização: expandir os itens do pedido
df_itens = json_normalize(
    data,
    # caminho dentro do JSON até a lista que você quer transformar em linhas da tabela.
    record_path=['pedido', 'itens'],
    # outras informações do nível superior que você também quer carregar junto
    meta=[['pedido', 'id'], ['pedido', 'cliente']]
)

print("Itens do pedido:")
df_itens.head()

Itens do pedido:


,produto,quantidade,precos,pedido.id,pedido.cliente
0,Notebook,1,"[3500, 3300]",303,Carlos
1,Mouse,2,"[80, 70]",303,Carlos


In [11]:
# 2ª normalização: explodir lista de preços (normal e promocional)

df_precos = df_itens.explode('precos').reset_index(drop=True)

print("\nItens com preços detalhados:")
df_precos.head()


Itens com preços detalhados:


,produto,quantidade,precos,pedido.id,pedido.cliente
0,Notebook,1,3500,303,Carlos
1,Notebook,1,3300,303,Carlos
2,Mouse,2,80,303,Carlos
3,Mouse,2,70,303,Carlos
